In [1]:
import math
import pandas_datareader as web
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import matplotlib.pyplot as plt
import requests
import random
from pprint import pprint 
import time
from sklearn.metrics import mean_squared_error
import json
import finnhub
plt.style.use('fivethirtyeight')

In [2]:
# Setup client
finnhub_client = finnhub.Client(api_key="bujelkv48v6righvsoc0")

In [4]:
lowes_earning_surprises = requests.get('https://finnhub.io/api/v1/stock/earnings?symbol=LOW&token=bujelkv48v6righvsoc0')
pprint(lowes_earning_surprises.json())

[{'actual': 3.75,
  'estimate': 2.9795000000000003,
  'period': '2020-09-30',
  'symbol': 'LOW'},
 {'actual': 1.77,
  'estimate': 1.3332000000000002,
  'period': '2020-06-30',
  'symbol': 'LOW'},
 {'actual': 0.94, 'estimate': 0.9191, 'period': '2020-03-31', 'symbol': 'LOW'},
 {'actual': 1.41,
  'estimate': 1.3635000000000002,
  'period': '2019-12-31',
  'symbol': 'LOW'}]


In [7]:
pprint(finnhub_client.company_basic_financials('LOW', 'margin'))

{'metric': {'freeOperatingCashFlow/revenue5Y': 3.98106,
            'freeOperatingCashFlow/revenueTTM': 11.33197,
            'grossMargin5Y': 32.87919,
            'grossMarginAnnual': 31.79991,
            'grossMarginTTM': 32.88636,
            'netProfitMargin5Y': 4.67119,
            'netProfitMarginAnnual': 5.93364,
            'netProfitMarginTTM': 7.12011,
            'operatingMargin5Y': 8.11234,
            'operatingMarginAnnual': 8.75146,
            'operatingMarginTTM': 10.52841,
            'pretaxMargin5Y': 7.17679,
            'pretaxMarginAnnual': 7.7937,
            'pretaxMarginTTM': 9.55319},
 'metricType': 'margin',
 'symbol': 'LOW'}


In [8]:
lowes_financial_estimates = finnhub_client.company_revenue_estimates('LOW', freq='quarterly')
pprint(lowes_financial_estimates)

{'data': [{'numberAnalysts': 2,
           'period': '2023-03-31',
           'revenueAvg': 18365430300,
           'revenueHigh': 18455900400,
           'revenueLow': 17551199400},
          {'numberAnalysts': 2,
           'period': '2022-12-31',
           'revenueAvg': 19566307250,
           'revenueHigh': 19662693000,
           'revenueLow': 18698835500},
          {'numberAnalysts': 2,
           'period': '2022-09-30',
           'revenueAvg': 24220254799,
           'revenueHigh': 24339566400,
           'revenueLow': 23146450400},
          {'numberAnalysts': 2,
           'period': '2022-06-30',
           'revenueAvg': 20803328349,
           'revenueHigh': 20905807800,
           'revenueLow': 19881013300},
          {'numberAnalysts': 12,
           'period': '2022-03-31',
           'revenueAvg': 18439200500,
           'revenueHigh': 19749546000,
           'revenueLow': 16373697000},
          {'numberAnalysts': 12,
           'period': '2021-12-31',
           'reve

In [9]:
recommendations = finnhub_client.recommendation_trends('LOW')
pprint(recommendations)

[{'buy': 12,
  'hold': 14,
  'period': '2020-10-01',
  'sell': 0,
  'strongBuy': 9,
  'strongSell': 1,
  'symbol': 'LOW'},
 {'buy': 17,
  'hold': 7,
  'period': '2020-09-01',
  'sell': 0,
  'strongBuy': 11,
  'strongSell': 0,
  'symbol': 'LOW'},
 {'buy': 16,
  'hold': 6,
  'period': '2020-08-01',
  'sell': 0,
  'strongBuy': 12,
  'strongSell': 0,
  'symbol': 'LOW'},
 {'buy': 11,
  'hold': 13,
  'period': '2020-07-01',
  'sell': 0,
  'strongBuy': 9,
  'strongSell': 1,
  'symbol': 'LOW'},
 {'buy': 11,
  'hold': 13,
  'period': '2020-06-01',
  'sell': 0,
  'strongBuy': 9,
  'strongSell': 1,
  'symbol': 'LOW'},
 {'buy': 16,
  'hold': 8,
  'period': '2020-05-01',
  'sell': 0,
  'strongBuy': 10,
  'strongSell': 0,
  'symbol': 'LOW'},
 {'buy': 11,
  'hold': 13,
  'period': '2020-04-01',
  'sell': 0,
  'strongBuy': 9,
  'strongSell': 1,
  'symbol': 'LOW'},
 {'buy': 11,
  'hold': 13,
  'period': '2020-03-01',
  'sell': 0,
  'strongBuy': 9,
  'strongSell': 1,
  'symbol': 'LOW'},
 {'buy': 11,
  '

In [12]:
# Stock splits
stock_splits = finnhub_client.stock_splits('LOW', _from='1990-01-01', to='2020-04-01')
pprint(stock_splits)

[{'date': '2006-07-03', 'fromFactor': 1, 'symbol': 'LOW', 'toFactor': 2},
 {'date': '2001-07-02', 'fromFactor': 1, 'symbol': 'LOW', 'toFactor': 2},
 {'date': '1998-06-29', 'fromFactor': 1, 'symbol': 'LOW', 'toFactor': 2},
 {'date': '1994-04-04', 'fromFactor': 1, 'symbol': 'LOW', 'toFactor': 2},
 {'date': '1992-06-29', 'fromFactor': 1, 'symbol': 'LOW', 'toFactor': 2}]


In [14]:
# Stock dividends
dividends = finnhub_client.stock_dividends('LOW', _from='1990-01-01', to='2020-04-01')
pprint(dividends)

[{'adjustedAmount': 0.55,
  'amount': 0.55,
  'currency': 'USD',
  'date': '2020-01-21',
  'declarationDate': '2019-11-08',
  'payDate': '2020-02-05',
  'recordDate': '2020-01-22',
  'symbol': 'LOW'},
 {'adjustedAmount': 0.55,
  'amount': 0.55,
  'currency': 'USD',
  'date': '2019-10-22',
  'declarationDate': '2019-08-15',
  'payDate': '2019-11-06',
  'recordDate': '2019-10-23',
  'symbol': 'LOW'},
 {'adjustedAmount': 0.55,
  'amount': 0.55,
  'currency': 'USD',
  'date': '2019-07-23',
  'declarationDate': '2019-05-31',
  'payDate': '2019-08-07',
  'recordDate': '2019-07-24',
  'symbol': 'LOW'},
 {'adjustedAmount': 0.48,
  'amount': 0.48,
  'currency': 'USD',
  'date': '2019-04-23',
  'declarationDate': '2019-03-22',
  'payDate': '2019-05-08',
  'recordDate': '2019-04-24',
  'symbol': 'LOW'},
 {'adjustedAmount': 0.48,
  'amount': 0.48,
  'currency': 'USD',
  'date': '2019-01-22',
  'declarationDate': '2018-11-09',
  'payDate': '2019-02-06',
  'recordDate': '2019-01-23',
  'symbol': 'LO